In [1]:
%load_ext autoreload

%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd

from library import *

In [3]:
db = DB()

In [4]:
f_invoice_payments = db.gettable('f_invoice_payments')

In [5]:
f_invoice_payments.head()

,id_invoices,uuid_payment_id,payment_created_at,payment_updated_at,amount,provider,state,charge_id,transfer_id,sincro_at,id_business
0,59f7546e3d496965e9c18b66,5a47562e6fb891690fe2b94f,20171230,20180109,59.29,gocardless,paid_out,PM001M4TEWBJHR,59f7546e3d496965e9c18b66,2018-07-12 21:54:52.567036+00:00,-1
1,59f7568e388b47657851c58b,5a4756406fb891690fe2b952,20171230,20180109,36.30,gocardless,paid_out,PM001M4TEYR5FF,59f7568e388b47657851c58b,2018-07-12 21:54:52.567036+00:00,53300
2,5a6cd1f039d60273ecf04f42,5a6cd31186e2c2373c1d3526,20180127,20180127,3.00,stripe,transfer,ch_1BozM5JahFDQqyZkgYN5WvuW,tr_1BozM7JahFDQqyZke7r45WMA,2018-07-12 21:54:52.567036+00:00,53305
3,5a6ccf6de4c1f8750a151f40,5a6cd496e4c1f8750a151f4f,20180127,20180127,3.00,stripe,transfer,ch_1BozSMJahFDQqyZkHjEedee9,tr_1BozSPJahFDQqyZk5MHZ9jnH,2018-07-12 21:54:52.567036+00:00,53305
4,5a6ce3e3e4c1f8750a151f58,5a6ce4b927fdb473f149f41b,20180127,20180127,3.00,stripe,succeeded,ch_1Bp0X0JahFDQqyZk2S0smHcd,tr_1Bp0X2JahFDQqyZkWgvnmtvQ,2018-07-12 21:54:52.567036+00:00,53305


In [6]:
pd.get_dummies(f_invoice_payments.state).head()

,cancelled,charged_back,confirmed,failed,paid,paid_out,pending_submission,submitted,succeeded,transfer
0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,0


In [7]:
pd.get_dummies(f_invoice_payments.provider).head()

,gocardless,stripe,unnax
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,0,1,0


In [8]:
f_invoice_payments['pay_delay'] = (pd.to_datetime(f_invoice_payments.payment_updated_at, yearfirst=True,format='%Y%m%d').astype(int) // 10 ** 9 - \
pd.to_datetime(f_invoice_payments.payment_created_at, yearfirst=True,format='%Y%m%d').astype(int) // 10 ** 9) // 86400

In [9]:
f_invoice_payments.drop(columns=['payment_created_at','payment_updated_at', 'transfer_id', 'charge_id', 'uuid_payment_id', 'sincro_at'], inplace=True)

In [10]:
f_invoice_payments = pd.concat([f_invoice_payments, pd.get_dummies(f_invoice_payments.provider), pd.get_dummies(f_invoice_payments.state)], axis=1)

In [11]:
f_invoice_payments.drop(columns=['provider','state'], inplace=True)

In [12]:
f_invoice_payments.head()

,id_invoices,amount,id_business,pay_delay,gocardless,stripe,unnax,cancelled,charged_back,confirmed,failed,paid,paid_out,pending_submission,submitted,succeeded,transfer
0,59f7546e3d496965e9c18b66,59.29,-1,10,1,0,0,0,0,0,0,0,1,0,0,0,0
1,59f7568e388b47657851c58b,36.30,53300,10,1,0,0,0,0,0,0,0,1,0,0,0,0
2,5a6cd1f039d60273ecf04f42,3.00,53305,0,0,1,0,0,0,0,0,0,0,0,0,0,1
3,5a6ccf6de4c1f8750a151f40,3.00,53305,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4,5a6ce3e3e4c1f8750a151f58,3.00,53305,0,0,1,0,0,0,0,0,0,0,0,0,1,0
